# Cheese Recommender

## EDA and Cleaning

In [1]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import unicodedata

Recommending cheese based on blah blah blah...  Cheese.com very lenient API

In [2]:
# took Adi Bronshtein's function for conducting exploratory data analysis and added spaces between each eda test
# using blank print statesments and \n for easier readability
def eda(dataframe):
    print("missing values: \n{}".format(dataframe.isnull().sum()))
    print()
    print("dataframe index: \n{}".format(dataframe.index))
    print()
    print("dataframe types: \n{}".format(dataframe.dtypes))
    print()
    print("dataframe shape: \n{}".format(dataframe.shape))
    print()
    print("dataframe describe: \n{}".format(dataframe.describe()))
    print()

    print("number of unique values in each column: ")
    for item in dataframe:
        print(item)
        print(dataframe[item].nunique())

In [3]:
# function to strip strings of accents, from BartoszKP and oefe on StackOverflow
# https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-in-a-python-unicode-string

import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')


In [4]:
#read culture_cheese_mag.csv into 'culture_chz'
culture_chz = pd.read_csv('./data/culture_cheese_mag_fixed.csv')


In [5]:
culture_chz

,Name,Producer,Country,Region,Size,Weight,Website,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description
0,Abbaye de Belloc,Abbaye de Notre-Dame de Belloc,France,Aquitaine,"8-9 in. diameter, 3-4 in. high",8-10 lbs,NaN,Sheep,Raw,Firm,Animal,"Natural, Paprika","Alpine-style, Cheddar-like",Flavor added to rind,Abbaye de Belloc is a French cheese that is ma...
1,Abbaye de Citeaux,Frère Joel & Frère Frédéric,France,Burgundy,8 in. diameter. 2 in. high,2 lbs,NaN,Cow,Raw,Semi Soft,Animal,"Washed, Mold Ripened","Soft-Ripened (Brie-like), Washed-Rind",NaN,"Located near Losne in Burgundy, France, Abbaye..."
2,Abbaye de Tamie,Monks of Tamie,France,Savoie,"8 in. diameter, 2-3 in. high",3 lbs,NaN,Cow,Raw,Semi Soft,Animal,Washed,Washed-Rind,NaN,"Made from raw cow`s milk, Abbaye de Tamie is p..."
3,Abbaye du Mont des Cats,Abbaye du Mont des Cats,France,Nord-Pas-de-Calais,"8 in. diameter, 4 in. high",4 lbs,NaN,Cow,NaN,Semi Soft,Vegetable,Washed,"Alpine-style, Washed-Rind",NaN,Abbaye du Mont des Cats is a cow`s milk cheese...
4,Abondance,Various,France,Haut Savoie,"2 ¾ in. tall, 15 ¾ in. diameter",15 - 26 lbs,www.fromageabondance.fr/en/,Cow,Raw,Firm,Animal,Natural,Alpine-style,NaN,Named after a valley in the French Alps where ...
5,Acapella,Andante Dairy,United States,California,Varies,Varies,www.andantedairy.com,Goat,Pasteurised,Soft,Microbial,"Mold Ripened, Ash coated",Soft-Ripened (Brie-like),NaN,"Established in 1999, Andante Dairy is owned an..."
6,Adams Reserve Cheddar,Great Lakes Cheese,United States,New York,NaN,40 lbs,www.adamsreserve.com,Cow,Pasteurised,Hard,Vegetable,Plasticoat,Cheddar-like,NaN,Adams Reserve is produced by Great Lakes Chees...
7,Adantino with Tarragon,Andante Dairy,United States,California,"3.5 in. long, 1.5 in. wide, 0.75 in. high",5 oz,www.andantedairy.com,Goat,Pasteurised,Soft,Microbial,Mold Ripened,Soft-Ripened (Brie-like),Flavor added to paste,"Established in 1999, Andante Dairy is owned an..."
8,Aged Cheddar (Grafton),Grafton Village Cheese,United States,Vermont,Varies,8 oz; 1 lb; 5 lbs; 10 lbs,http://www.graftonvillagecheese.com/,Cow,Raw,Firm,Microbial,"None, Plastic",Cheddar-like,NaN,"Located in Vermont, the Grafton Cooperative Ch..."
9,Airco,Carr Valley Cheese Co,United States,Wisconsin,"8 ins diameter, 4 ins high",10 lbs,www.carrvalleycheese.com,"Goat, Sheep, Cow",Pasteurised,Semi Soft,Vegetable,Smoked,"Alpine-style, Cheddar-like",Smoked,Located in the rolling hills of central Wiscon...


In [6]:
# it looks like the info given in the 'Flavor' column in this data set is useless except for telling whether or
# not the cheese is smoked, so I decided to extract whether a cheese was smoked or not, make a new column out
# of that info (called 'Smoked'), and then drop the Flavor column
culture_chz['Flavor'].unique()

array(['Flavor added to rind', nan, 'Flavor added to paste', 'Smoked',
       'Flavor added to rind, Smoked',
       'Flavor added to paste, Flavor added to rind',
       'Flavor added to paste, Smoked'], dtype=object)

In [7]:
culture_chz[culture_chz['Flavor'].notnull()]

,Name,Producer,Country,Region,Size,Weight,Website,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description
0,Abbaye de Belloc,Abbaye de Notre-Dame de Belloc,France,Aquitaine,"8-9 in. diameter, 3-4 in. high",8-10 lbs,NaN,Sheep,Raw,Firm,Animal,"Natural, Paprika","Alpine-style, Cheddar-like",Flavor added to rind,Abbaye de Belloc is a French cheese that is ma...
7,Adantino with Tarragon,Andante Dairy,United States,California,"3.5 in. long, 1.5 in. wide, 0.75 in. high",5 oz,www.andantedairy.com,Goat,Pasteurised,Soft,Microbial,Mold Ripened,Soft-Ripened (Brie-like),Flavor added to paste,"Established in 1999, Andante Dairy is owned an..."
9,Airco,Carr Valley Cheese Co,United States,Wisconsin,"8 ins diameter, 4 ins high",10 lbs,www.carrvalleycheese.com,"Goat, Sheep, Cow",Pasteurised,Semi Soft,Vegetable,Smoked,"Alpine-style, Cheddar-like",Smoked,Located in the rolling hills of central Wiscon...
19,Ancho Goat Caciotta,Mozzarella Company,United States,Texas,7.5 in diameter x 2 in height,5 lbs (large) and 1.5 lbs (mini),www.mozzarellaco.com,Goat,Pasteurised,Semi Soft,Animal,Waxed,NaN,Flavor added to paste,"Located in Dallas, Texas, the Mozzarella Compa..."
22,Anthill,Woodside Cheese Wrights,Australia,South Australia,NaN,NaN,woodsidecheese.com.au,Goat,Pasteurised,Soft,NaN,Flavor Added,Fresh,Flavor added to rind,America’s most adventurous turophiles have eag...
25,Apple Smoked Cheddar,Carr Valley Cheese Company,United States,Wisconsin,"10 ins diameter, 4 ins high",12lb,www.carrvalleycheese.com,Cow,NaN,Firm,Vegetable,"Flavored, Smoked, Paprika",Cheddar-like,"Flavor added to rind, Smoked",Located in the rolling hills of central Wiscon...
26,Appletree Smoked Cheddar,Cows Creamery,Canada,Prince Edward Island,24 in x 24 in x 10 in,40 lbs,www.cowscreamery.ca,Cow,Raw,Hard,Vegetable,Smoked,Cheddar-like,Smoked,Cows Creamery is located on Canada’s east cost...
54,Barely Buzzed,Beehive Cheese Company,United States,Utah,"12-14 ins diameter, 3 ins high",18 lbs,www.beehivecheese.com,Cow,Pasteurised,Firm,Microbial,Flavored,Cheddar-like,Flavor added to rind,Ever wondered what it would be like to blend c...
59,Baronerosso al Barbera,La Casearia Carpenedo,Italy,Veneto,12 in diameter x 4 in height,16 lbs,www.lacasearia.com,Goat,Pasteurised,Firm,Animal,Flavor Added,"Cheddar-like, Washed-Rind",Flavor added to rind,La Casearia is a creamery and aging facility i...
62,Basil Mascarpone Torta,Mozzarella Company,United States,Texas,3 in x 3 in,0.5 lbs,www.mozzco.com,Cow,Pasteurised,NaN,Vegetable,None,Fresh,Flavor added to paste,"Located in Dallas, Texas, the Mozzarella Compa..."


In [8]:
# because NaN's read like floats, they mess up the loop, so I had to convert them to strings
culture_chz['Flavor'].fillna('Unflavored', inplace=True)

In [9]:
'Smoked' in 'Flavor added to rindSmoked'

True

In [10]:
list = []
for i in range(len(culture_chz)):
    if "Smoked" in culture_chz['Flavor'][i]:
        name = culture_chz['Name'][i]
        list.append(name)
list

['Airco',
 'Apple Smoked Cheddar',
 'Appletree Smoked Cheddar',
 'Braided String Cheese - Hickory Smoked (Karoun)',
 'Cascaval',
 'Chechil',
 'Cheddar (Gold Creek Smoked White)',
 'Cheddar (Quickes Smoked)',
 'Cheddar (Shelburne Smoked)',
 'Frère Fumant',
 'Jersey Jack',
 'Maple Smoked Cheddar (Grafton Village)',
 'Scamorza - Smoked (The Mozzarella Company)',
 'Smoked Baby Swiss',
 'Smoked Chili Cheddar (Grafton)',
 'Smoked Gouda (Brazos Valley)',
 'Smoked Kentucky Bleu',
 'Smoked Parmesan (Gold Creek)',
 'Smoked Provolone (Silvery Moon Creamery)',
 'Smoking Goud',
 'Up in Smoke',
 'Up North Smoked Ricotta']

In [11]:
culture_chz.iloc[0]["Flavor"]

'Flavor added to rind'

In [12]:
culture_chz['Smoked'] = 0
for i in range(len(culture_chz)):
    if "Smoked" in culture_chz['Flavor'][i]:
        culture_chz['Smoked'][i] = 1
    else:
        culture_chz['Smoked'][i] = 0

/Users/michaelknight/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/michaelknight/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [13]:
# dont do this
# drop Flavor column
#culture_chz.drop(columns=['Flavor'], inplace=True)

In [14]:
# drop other features that are useless to this project
culture_chz.drop(columns=['Size', 'Website', 'Weight'], inplace=True)

In [15]:
culture_chz[culture_chz['Smoked'] == 1]

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description,Smoked
9,Airco,Carr Valley Cheese Co,United States,Wisconsin,"Goat, Sheep, Cow",Pasteurised,Semi Soft,Vegetable,Smoked,"Alpine-style, Cheddar-like",Smoked,Located in the rolling hills of central Wiscon...,1
25,Apple Smoked Cheddar,Carr Valley Cheese Company,United States,Wisconsin,Cow,NaN,Firm,Vegetable,"Flavored, Smoked, Paprika",Cheddar-like,"Flavor added to rind, Smoked",Located in the rolling hills of central Wiscon...,1
26,Appletree Smoked Cheddar,Cows Creamery,Canada,Prince Edward Island,Cow,Raw,Hard,Vegetable,Smoked,Cheddar-like,Smoked,Cows Creamery is located on Canada’s east cost...,1
135,Braided String Cheese - Hickory Smoked (Karoun),Karoun Dairies,United States,California,Cow,NaN,"Flavor Added, Semi Soft",Vegetable,None,Pasta Filata (mozzarella-type),Smoked,"Karoun Dairies, based in Sun Valley, CA, speci...",1
224,Cascaval,Various,Romania,Various,"Sheep, Cow","Pasteurised, Raw",Firm,NaN,"Smoked, Waxed",Pasta Filata (mozzarella-type),Smoked,Cascaval is a Romanian cheese that was traditi...,1
256,Chechil,Chechil USA,United States,New York,Cow,Pasteurised,Firm,Vegetable,Smoked,Pasta Filata (mozzarella-type),Smoked,"On their travels in Russia, Armenia and Centra...",1
278,Cheddar (Gold Creek Smoked White),Gold Creek Farms,United States,Utah,Cow,Pasteurised,Hard,Microbial,Smoked,Cheddar-like,Smoked,"In 2007, Alan and Debbie Gold purchased a 130-...",1
287,Cheddar (Quickes Smoked),Quickes Traditional Cheddar,United Kingdom,Devon,Cow,NaN,Firm,Animal,"Natural, Smoked",Cheddar-like,Smoked,"Home Farm, belonging to the Quicke family, is ...",1
291,Cheddar (Shelburne Smoked),Shelburne Farms,United States,Vermont,Cow,Pasteurised,Firm,Microbial,Smoked,Cheddar-like,Smoked,Located in a stunning position on the shores o...,1
526,Frère Fumant,3 Corner Field Farm,NaN,New York,Sheep,NaN,Firm,Animal,Smoked,NaN,Smoked,Owned by Karen Weinberg and her husband Paul B...,1


In [16]:
culture_chz

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description,Smoked
0,Abbaye de Belloc,Abbaye de Notre-Dame de Belloc,France,Aquitaine,Sheep,Raw,Firm,Animal,"Natural, Paprika","Alpine-style, Cheddar-like",Flavor added to rind,Abbaye de Belloc is a French cheese that is ma...,0
1,Abbaye de Citeaux,Frère Joel & Frère Frédéric,France,Burgundy,Cow,Raw,Semi Soft,Animal,"Washed, Mold Ripened","Soft-Ripened (Brie-like), Washed-Rind",Unflavored,"Located near Losne in Burgundy, France, Abbaye...",0
2,Abbaye de Tamie,Monks of Tamie,France,Savoie,Cow,Raw,Semi Soft,Animal,Washed,Washed-Rind,Unflavored,"Made from raw cow`s milk, Abbaye de Tamie is p...",0
3,Abbaye du Mont des Cats,Abbaye du Mont des Cats,France,Nord-Pas-de-Calais,Cow,NaN,Semi Soft,Vegetable,Washed,"Alpine-style, Washed-Rind",Unflavored,Abbaye du Mont des Cats is a cow`s milk cheese...,0
4,Abondance,Various,France,Haut Savoie,Cow,Raw,Firm,Animal,Natural,Alpine-style,Unflavored,Named after a valley in the French Alps where ...,0
5,Acapella,Andante Dairy,United States,California,Goat,Pasteurised,Soft,Microbial,"Mold Ripened, Ash coated",Soft-Ripened (Brie-like),Unflavored,"Established in 1999, Andante Dairy is owned an...",0
6,Adams Reserve Cheddar,Great Lakes Cheese,United States,New York,Cow,Pasteurised,Hard,Vegetable,Plasticoat,Cheddar-like,Unflavored,Adams Reserve is produced by Great Lakes Chees...,0
7,Adantino with Tarragon,Andante Dairy,United States,California,Goat,Pasteurised,Soft,Microbial,Mold Ripened,Soft-Ripened (Brie-like),Flavor added to paste,"Established in 1999, Andante Dairy is owned an...",0
8,Aged Cheddar (Grafton),Grafton Village Cheese,United States,Vermont,Cow,Raw,Firm,Microbial,"None, Plastic",Cheddar-like,Unflavored,"Located in Vermont, the Grafton Cooperative Ch...",0
9,Airco,Carr Valley Cheese Co,United States,Wisconsin,"Goat, Sheep, Cow",Pasteurised,Semi Soft,Vegetable,Smoked,"Alpine-style, Cheddar-like",Smoked,Located in the rolling hills of central Wiscon...,1


In [17]:
eda(culture_chz)

missing values: 
Name                0
Producer           31
Country            35
Region             12
Milk                5
Treatment         680
Classification     19
Rennet             45
Rind               30
Style             703
Flavor              0
Description         0
Smoked              0
dtype: int64

dataframe index: 
RangeIndex(start=0, stop=1244, step=1)

dataframe types: 
Name              object
Producer          object
Country           object
Region            object
Milk              object
Treatment         object
Classification    object
Rennet            object
Rind              object
Style             object
Flavor            object
Description       object
Smoked             int64
dtype: object

dataframe shape: 
(1244, 13)

dataframe describe: 
            Smoked
count  1244.000000
mean      0.017685
std       0.131856
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max       1.000000

number of unique values in each column: 
Nam

In [18]:
culture_chz['Rennet'].unique()

array(['Animal', 'Vegetable', 'Microbial', nan, 'True Vegetable',
       'Animal, Microbial', 'Chymosin', 'Vegetable, Animal'], dtype=object)

In [19]:
culture_chz[culture_chz['Style'].isnull()]

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description,Smoked
19,Ancho Goat Caciotta,Mozzarella Company,United States,Texas,Goat,Pasteurised,Semi Soft,Animal,Waxed,NaN,Flavor added to paste,"Located in Dallas, Texas, the Mozzarella Compa...",0
29,Arseniko Naxou,Various,Greece,Naxos,"Sheep, Cow, Goat","Pasteurised, Raw",Firm,NaN,"Natural, Flavor Added",NaN,Unflavored,Hailing from the mountain village of Koronos o...,0
37,Assa,Tomales Farmstead Creamery,United States,California,Goat,Pasteurised,Firm,Animal,"Natural, Flavor Added",NaN,Unflavored,Husband-and-wife duo David Jablons and Tamara...,0
38,Atika,Tomales Farmstead Creamery,United States,California,"Sheep, Goat",Pasteurised,Firm,Animal,"Natural, Flavor Added",NaN,Unflavored,Husband-and-wife duo David Jablons and Tamara...,0
90,Big Ed's,Saxon Homestead Creamery,NaN,Wisconsin,Cow,Raw,Firm,Microbial,Washed,NaN,Unflavored,"Located just north of Milwaukee, close to the ...",0
91,Big John's Cajun,Beehive Cheese Company,United States,Utah,Cow,Pasteurised,Firm,Microbial,Flavored,NaN,Unflavored,Founded in 2005 by brothers-in-law Tim Welsh a...,0
96,Black and Bleu,Winter Park Dairy,United States,Florida,Cow,Raw,Firm,Microbial,Natural,NaN,Unflavored,Owned and operated by the Green family for 4 g...,0
102,Blackstone,Bellwether Farms,United States,California,"Sheep, Cow",Pasteurised,Hard,Animal,Flavor Added,NaN,Flavor added to rind,Named after the volcanic outcroppings that su...,0
118,Blue Ridge Bleu,Blue Ridge Creamery,United States,South Carolina,Cow,Raw,Blue,NaN,Natural,NaN,Unflavored,"In the sleepy town of Travelers Rest, South C...",0
120,Bluebird,The Grey Barn and Farm,United States,Massachusetts,Cow,Raw,Blue,NaN,Natural,NaN,Unflavored,"“A wistful note from out the sky/`pure, pure, ...",0


In [20]:
culture_chz['Style'].unique()

array(['Alpine-style, Cheddar-like',
       'Soft-Ripened (Brie-like), Washed-Rind', 'Washed-Rind',
       'Alpine-style, Washed-Rind', 'Alpine-style',
       'Soft-Ripened (Brie-like)', 'Cheddar-like',
       'Alpine-style, Gouda-like, Washed-Rind',
       'Washed-Rind, Alpine-style', 'Blue', 'Grana / Grating cheese', nan,
       'Fresh', 'Alpine-style, Soft-Ripened (Brie-like)',
       'Pasta Filata (mozzarella-type)',
       'Alpine-style, Cheddar-like, Grana / Grating cheese',
       'Crème, Washed-Rind',
       'Cheddar-like, Gouda-like, Grana / Grating cheese, Washed-Rind',
       'Blue, Cheddar-like', 'Gouda-like, Grana / Grating cheese',
       'Cheddar-like, Washed-Rind',
       'Cheddar-like, Grana / Grating cheese',
       'Blue, Soft-Ripened (Brie-like)',
       'Alpine-style, Cheddar-like, Gouda-like', 'Alpine-style, Blue',
       'Gouda-like', 'Crème, Soft-Ripened (Brie-like)',
       'Fresh, Pasta Filata (mozzarella-type)', 'Crème', 'Blue, Crème',
       'Alpine-style, G

In [21]:
culture_chz['Treatment'].unique()

array(['Raw', nan, 'Pasteurised', 'Heat Treated', 'Pasteurised, Raw',
       'Thermalised'], dtype=object)

In [22]:
culture_chz['Treatment'].unique()

array(['Raw', nan, 'Pasteurised', 'Heat Treated', 'Pasteurised, Raw',
       'Thermalised'], dtype=object)

In [23]:
culture_chz['Classification'].unique()

array(['Firm', 'Semi Soft', 'Soft', 'Hard', 'Blue, Semi Soft',
       'Blue, Firm', 'Firm, Hard', 'Semi Soft, Blue', 'Firm, Blue',
       'Blue', nan, 'Flavor Added, Semi Soft', 'Blue, Soft',
       'Flavor Added', 'Soft, Flavor Added', 'Semi Soft, Firm',
       'Semi Soft, Firm, Hard', 'Flavor Added, Firm',
       'Semi Soft, Flavor Added', 'Firm, Flavor Added'], dtype=object)

In [24]:
# since there are only 5 cheeses in this list with missing Milk type, I decided to manually look them up!
culture_chz[culture_chz['Milk'].isnull()]

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description,Smoked
518,Fromage Blanc (Cowgirl Creamery),Cowgirl Creamery,United States,California,NaN,NaN,Soft,Microbial,None,NaN,Unflavored,"Cowgirl Creamery, based in Marin County, Calif...",0
761,Melkbus125 Winter,Uniekaas with De Producent,Netherlands,Gouda,NaN,NaN,NaN,NaN,NaN,NaN,Unflavored,The Melkbus line of cheeses represent a collab...,0
956,Ricotta (Caputo Brothers),Caputo Brother’s Creamery,United States,Pennsylvania,NaN,NaN,Soft,NaN,None,NaN,Unflavored,"Owned and operate by Rynn and David Caputo, an...",0
966,Ricotta Salata (BelGioioso),BelGioioso Cheese Co.,United States,Wisconsin,NaN,NaN,Semi Soft,NaN,None,NaN,Unflavored,"Founded by Errico Auricchio, BelGioioso Cheese...",0
968,Riesling x Sylvaner,BonCas Dairy,Switzerland,Dotzigen,NaN,NaN,Soft,Animal,Mold Ripened,NaN,Unflavored,Riesling x Sylvaner is made by Ueli Moser at h...,0


In [25]:
mask = culture_chz['Name'] == 'Fromage Blanc (Cowgirl Creamery)'

culture_chz[mask] = culture_chz[mask].fillna({'Milk' :'Cow', 'Treatment': 'Pasteurised'})

#culture_chz[culture_chz['Name'] == 'Fromage Blanc (Cowgirl Creamery)'].iloc[0][['Milk']] = culture_chz[culture_chz['Name'] == 'Fromage Blanc (Cowgirl Creamery)'].iloc[0][['Milk']].fillna("Cow")







In [26]:
culture_chz[culture_chz['Name'] == 'Fromage Blanc (Cowgirl Creamery)']

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description,Smoked
518,Fromage Blanc (Cowgirl Creamery),Cowgirl Creamery,United States,California,Cow,Pasteurised,Soft,Microbial,None,NaN,Unflavored,"Cowgirl Creamery, based in Marin County, Calif...",0


In [27]:
# found info on Cowgirl Creamery's Fromage Blanc by googling it (fun fact:  i used to work there in 2008!)
# found not only the milk type but Treatment info as well (it's pasteurized)
# https://www.cowgirlcreamery.com/library-of-cheese/fromage-blanc

mask = culture_chz['Name'] == 'Fromage Blanc (Cowgirl Creamery)'

culture_chz[mask] = culture_chz[mask].fillna({'Milk' :'Cow', 'Treatment': 'Pasteurised'})



In [28]:
# found info on Melkbus125 Winter by googling it 
# found not only the milk type but Treatment info as well (it's raw), and rennet type
# since it's gouda can deduce that style is gouda-like and classification is firm
# http://uniekaasusa.com/cheeses/melkbus-winter-2/


mask = culture_chz['Name'] == 'Melkbus125 Winter'

culture_chz[mask] = culture_chz[mask].fillna({'Milk' :'Cow', 'Treatment': 'Pasteurised', 'Rennet' :'Animal', 
                                              'Style': 'Gouda-like', 'Classification': 'Firm'})


In [29]:
# found info on Ricotta (Caputo Brothers) by googling it 
# found not only the milk type but Treatment, Style and Rennet
# https://caputobrotherscreamery.com/product/whey-based-ricotta/

mask = culture_chz['Name'] == 'Ricotta (Caputo Brothers)'

culture_chz[mask] = culture_chz[mask].fillna({'Milk' :'Cow', 'Treatment': 'Pasteurised', 'Rennet' :'Animal', 
                                              'Style': 'Fresh'})

In [30]:
# found info on Ricotta Salata (BelGioioso) by googling it 
# found not only the milk type but Treatment, Style, and Rennet
# https://www.belgioioso.com/Products/Ricotta-Salata

mask = culture_chz['Name'] == 'Ricotta Salata (BelGioioso)'

culture_chz[mask] = culture_chz[mask].fillna({'Milk' :'Cow', 'Treatment': 'Pasteurised', 'Rennet' :'Vinegar', 
                                              'Style': 'Feta-like'})

In [31]:
# found info on Riesling x Sylvaner by googling it 
# found not only the milk type but Treatment info as well (it's raw)
# https://caputobrotherscreamery.com/product/whey-based-ricotta/


mask = culture_chz['Name'] == 'Riesling x Sylvaner'

culture_chz[mask] = culture_chz[mask].fillna({'Milk' :'Cow', 'Treatment': 'Pasteurised', 
                                              'Style': 'Soft-Ripened (Brie-like)'})

In [32]:
eda(culture_chz)

missing values: 
Name                0
Producer           31
Country            35
Region             12
Milk                0
Treatment         675
Classification     18
Rennet             42
Rind               30
Style             699
Flavor              0
Description         0
Smoked              0
dtype: int64

dataframe index: 
RangeIndex(start=0, stop=1244, step=1)

dataframe types: 
Name              object
Producer          object
Country           object
Region            object
Milk              object
Treatment         object
Classification    object
Rennet            object
Rind              object
Style             object
Flavor            object
Description       object
Smoked             int64
dtype: object

dataframe shape: 
(1244, 13)

dataframe describe: 
            Smoked
count  1244.000000
mean      0.017685
std       0.131856
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max       1.000000

number of unique values in each column: 
Nam

In [33]:
len(culture_chz)

1244

In [34]:
for i in culture_chz['Name']:
    if "(" in i:
        print(i)


Aged Cheddar (Grafton)
Alpine Style (Kennebec)
Asiago (Sartori)
Baby Swiss (Boggy Meadow)
Balsamic Goat (Ceased Production)
Bartlett Blue (Ceased Production)
Beermat (Aarauer Bierdeckel, Kuentener)
Blue Cheese Crumbles (Litehouse)
Blue Cheese (Litehouse)
Braided String Cheese - Hickory Smoked (Karoun)
Braided String Cheese - Olive, Garlic (Karoun)
Braided String Cheese - Original (Karoun)
Braided String Cheese with Black Caraway (Karoun)
Bread Cheese (Carr Valley)
Brebis (Blackberry Farms)
Brie (Brazos Valley)
Brie (Marin French Cheese)
Bucheron (Montchevré)
Buchette (Ash-rinded)
Buffalo Mozzarella (Deca & Otto)
Burrata (Belgioioso)
Burrata (Deca & Otto)
Burrata (Maplebrook Farm)
Burrata (Mozzarella Co)
Cabécou (Andante Dairy)
Caerphilly (Gorwydd Farm)
Caerphilly (Tonjes)
Camellia (Discontinued)
Camembert (Le Chatelain)
Camembert (Silvery Moon Creamery)
Caspian (Ceased Production)
Cave Aged Cheddar (Carr Valley)
Chabis (Laura Chenel)
Aged Cheddar (Kerrygold)
Cheddar (Avonlea)
Cheddar (

In [35]:
culture_chz['Milk'].unique() #if milk type is a list, that means it's a blend.  can we merge the lists that are
# the same animals but in different orders at least?

array(['Sheep', 'Cow', 'Goat', 'Goat, Sheep, Cow', 'Sheep, Cow, Goat',
       'Sheep, Goat', 'Buffalo', 'Sheep, Cow', 'Goat, Cow', 'Goat, Sheep',
       'Cow, Goat', 'Sheep, Goat, Cow', 'Buffalo, Cow', 'Cow, Sheep',
       'Cow, Buffalo'], dtype=object)

In [36]:
culture_chz[culture_chz['Milk'] == 'Goat, Sheep, Cow']

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description,Smoked
9,Airco,Carr Valley Cheese Co,United States,Wisconsin,"Goat, Sheep, Cow",Pasteurised,Semi Soft,Vegetable,Smoked,"Alpine-style, Cheddar-like",Smoked,Located in the rolling hills of central Wiscon...,1
77,Benedictine,Carr Valley Cheese Co,United States,Wisconsin,"Goat, Sheep, Cow",NaN,Semi Soft,Vegetable,None,"Alpine-style, Cheddar-like, Gouda-like",Unflavored,Located in the rolling hills of central Wiscon...,0
132,Bra Duro Vecchio,Various,Italy,"Cuneo, Piedmont","Goat, Sheep, Cow",NaN,Hard,Animal,Natural,NaN,Unflavored,Produced in the lowland dairies surrounding th...,0
1129,Three Milk Cenere,Affineur: Guffanti,Italy,Puglia,"Goat, Sheep, Cow",NaN,Soft,Animal,Ash coated,NaN,Unflavored,"Founded by Luigi Guffanti in 1876, Guffanti Fo...",0
1130,Three Sisters,Nettle Meadow Farm,United States,New York,"Goat, Sheep, Cow",NaN,Semi Soft,Vegetable,Mold Ripened,NaN,Unflavored,"Located in Thurman, New York, near the near th...",0
1166,Tres Leches,Rio Deva Dairy,Spain,Cantabria,"Goat, Sheep, Cow",NaN,Semi Soft,Animal,Mold Ripened,NaN,Unflavored,Located on the Deva River in the Picos de Euro...,0


In [37]:
culture_chz[culture_chz['Milk'] == 'Sheep, Cow, Goat']

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description,Smoked
29,Arseniko Naxou,Various,Greece,Naxos,"Sheep, Cow, Goat","Pasteurised, Raw",Firm,NaN,"Natural, Flavor Added",NaN,Unflavored,Hailing from the mountain village of Koronos o...,0
147,Briar Summit,Nettle Meadow Farm and Artisan Cheese,United States,New York,"Sheep, Cow, Goat",Pasteurised,Semi Soft,Microbial,Mold Ripened,Soft-Ripened (Brie-like),Flavor added to paste,This mountain-shaped cheese appropriately hail...,0
228,Caso Bolo Mellage,Carr Valley Cheese Co,United States,Wisconsin,"Sheep, Cow, Goat",NaN,Firm,Vegetable,Waxed,NaN,Unflavored,Located in the rolling hills of central Wiscon...,0
572,Gran Canaria,Carr Valley Cheese Company,United States,Wisconsin,"Sheep, Cow, Goat",NaN,Firm,Vegetable,Plastic,NaN,Unflavored,Located in the rolling hills of central Wiscon...,0
579,Graviera,Various,Greece,Various,"Sheep, Cow, Goat",Raw,Firm,NaN,NaN,Grana / Grating cheese,Unflavored,"This yellow, hard, buttery, and aromatic table...",0
646,Iberico,Various,Spain,"Castile-La Mancha, Castilla Leon","Sheep, Cow, Goat",NaN,Firm,Animal,Plastic,NaN,Unflavored,"Similar to Manchego, Iberico is made from a bl...",0
682,La Tur,Caseificio dell'Alta Langa,Italy,Alba,"Sheep, Cow, Goat",NaN,Soft,Animal,Mold Ripened,NaN,Unflavored,"Located in Bosia, between Alba and Cortemilla ...",0
763,Menage,Carr Valley Cheese Company,United States,Wisconsin,"Sheep, Cow, Goat",NaN,Firm,Vegetable,Waxed,NaN,Unflavored,Located in the rolling hills of central Wiscon...,0
978,Robiola Vite (Cora),Cora,Italy,Piedmont,"Sheep, Cow, Goat",NaN,Soft,Animal,Leaf Wrapped,NaN,Unflavored,"Piedmont, which lies in the Alpine foothills o...",0
980,Rochetta,Caseificio dell'Alta Langa,Italy,Piedmont,"Sheep, Cow, Goat",NaN,Soft,Animal,Mold Ripened,NaN,Unflavored,"Located in Bosia, between Alba and Cortemilla ...",0


In [38]:
culture_chz[culture_chz['Milk'] == 'Sheep, Goat']

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description,Smoked
38,Atika,Tomales Farmstead Creamery,United States,California,"Sheep, Goat",Pasteurised,Firm,Animal,"Natural, Flavor Added",NaN,Unflavored,Husband-and-wife duo David Jablons and Tamara...,0
51,Banon,Various,France,"Banon, Provence","Sheep, Goat",NaN,Soft,Animal,"Leaf Wrapped, Washed",Fresh,Unflavored,"Named after the market town, just north of Aix...",0
164,Brocciu,Various,France,Corsica,"Sheep, Goat",NaN,Soft,Animal,NaN,NaN,Unflavored,"According to Corsican legend, there was once a...",0
298,Cherbourg,Baetje Farms,United States,Missouri,"Sheep, Goat",NaN,Semi Soft,Animal,Washed,NaN,Unflavored,Steve and Veronica Baetje have been making che...,0
337,Coeur du Clos,Baetje Farms,United States,Missouri,"Sheep, Goat",Pasteurised,Semi Soft,Animal,Mold Ripened,Soft-Ripened (Brie-like),Unflavored,Steve and Veronica Baetje have been making che...,0
466,"Feta (Mt Vikos, Barrel Aged)",NaN,Greece,Thessaly,"Sheep, Goat",NaN,Firm,Animal,Natural,NaN,Unflavored,Production of Mt. Vikos Barrel Aged Feta takes...,0
468,Feta PDO,Various,Greece,Various,"Sheep, Goat","Pasteurised, Raw",Firm,Animal,Rindless,NaN,Unflavored,The poster child of Mediterranean brined chees...,0
597,Halloumi,Various,Cyprus,NaN,"Sheep, Goat",NaN,Firm,Animal,Rindless,NaN,Unflavored,"Halloumi is a bit modest-looking, with a spong...",0
767,Miette,Baetje Farms,United States,Missouri,"Sheep, Goat",Pasteurised,Semi Soft,Animal,Mold Ripened,NaN,Unflavored,Steve and Veronica Baetje have been making ch...,0
1215,Vivace,Andante Dairy,United States,California,"Sheep, Goat",NaN,Soft,Microbial,Mold Ripened,NaN,Unflavored,"Established in 1999, Andante Dairy is owned an...",0


In [39]:
culture_chz['Description'][1215]

'Established in 1999, Andante Dairy is owned and operated by Soyoung Scanlan. The creamery is located at the Volpi Ranch near Petaluma, just north of San Francisco, California, which is also the source for Scanlan`s goat`s milk. Cow`s milk is brought in from the 400-strong herd of Jersey cows at nearby Spring Hill Dairy. Prior to becoming a cheesemaker, Soyoung had a successful career working as a biochemist in both the engineering and science worlds. She also has a strong background in classical music as an avid pianist, making her a rare combination of talents that seem to converge over the cheesemaking vat. Having studied the properties of milk and cheesemaking intensely for two years, including a stint at Cal Poly, in San Luis Obispo, Soyoung has become a highly respected figure in the artisan cheese industry. She is known for producing small quantities of extremely high quality cheeses, many of which are made in the French style. By choice, Soyoung works alone. Her cheesemaking fa

In [40]:
culture_chz[culture_chz['Milk'] == 'Sheep, Cow']

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description,Smoked
57,Barn Dance,Grafton Village Cheese,United States,Vermont,"Sheep, Cow",Raw,"Firm, Hard",Microbial,None,Alpine-style,Unflavored,"Located in Vermont, the Grafton Cooperative Ch...",0
102,Blackstone,Bellwether Farms,United States,California,"Sheep, Cow",Pasteurised,Hard,Animal,Flavor Added,NaN,Flavor added to rind,Named after the volcanic outcroppings that su...,0
224,Cascaval,Various,Romania,Various,"Sheep, Cow","Pasteurised, Raw",Firm,NaN,"Smoked, Waxed",Pasta Filata (mozzarella-type),Smoked,Cascaval is a Romanian cheese that was traditi...,1
318,Ciabot,Cora,Italy,Piedmont,"Sheep, Cow",NaN,Soft,Animal,Mold Ripened,NaN,Unflavored,"Piedmont, which lies in the Alpine foothills o...",0
356,Cowbilly,Natural Valley Cheese,United States,Wisconsin,"Sheep, Cow",NaN,Hard,Vegetable,Natural,NaN,Unflavored,Cheesemaker Tom Torkelson makes artisan cheese...,0
362,Crema de Blue,Valley Shepherd Creamery,United States,New Jersey,"Sheep, Cow",Raw,Semi Soft,Animal,Natural,Blue,Unflavored,"Located in Long Valley, New Jersey, Valley She...",0
452,Feliciana Nevat,Bittersweet Plantation Dairy,United States,Louisiana,"Sheep, Cow",NaN,Semi Soft,Vegetable,Mold Ripened,NaN,Unflavored,Bittersweet Plantation Dairy is owned by renow...,0
539,Gatekeeper,Old Chatham Sheepherding Company / Crown Finis...,United States,New York,"Sheep, Cow",Pasteurised,Semi Soft,NaN,Mold Ripened,Soft-Ripened (Brie-like),Flavor added to rind,This triple-cream cheese starts out young disk...,0
621,Hidden Falls,Shepherd's Way Farms,United States,Minnesota,"Sheep, Cow",NaN,Soft,Vegetable,Mold Ripened,NaN,Unflavored,"Located in south eastern Minnesota, Shepherd`s...",0
754,Meadow Melody Grande,Hidden Springs Creamery,United States,Wisconsin,"Sheep, Cow",Raw,Firm,Animal,Washed,NaN,Unflavored,"You can only find this buttery, washed-rind ch...",0


In [41]:
culture_chz[culture_chz['Milk'] == 'Cow, Goat']

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description,Smoked
124,Bon Pere,Boston Post Dairy,United States,Vermont,"Cow, Goat",Pasteurised,Firm,Microbial,Waxed,Gouda-like,Unflavored,Family-run Boston Post Dairy is set in Enosbur...,0
215,Capriella,Mozzarella Company,United States,Texas,"Cow, Goat",Pasteurised,Semi Soft,Vegetable,None,Pasta Filata (mozzarella-type),Unflavored,"Located in Dallas, Texas, the Mozzarella Compa...",0
239,Cella,Mozzarella Company,United States,Texas,"Cow, Goat",Pasteurised,Semi Soft,Vegetable,Leaf Wrapped,Soft-Ripened (Brie-like),Flavor added to rind,"Located in Dallas, Texas, the Mozzarella Compa...",0
247,Chandoka,LaClare Farms,United States,Wisconsin,"Cow, Goat",Pasteurised,Hard,Microbial,Cloth Wrapped,Cheddar-like,Unflavored,"Freshly mown grass, pickle relish, vegetal no...",0
365,Cremont,Vermont Creamery,United States,Vermont,"Cow, Goat",NaN,Soft,Microbial,Geotricum,NaN,Unflavored,Founded in 1984 by Allison Hooper and Bob Rees...,0
419,Echo Mountain Blue,Rogue Creamery,United States,Oregon,"Cow, Goat",Pasteurised,"Blue, Firm",Vegetable,Natural,Blue,Unflavored,"With 10 blue cheeses under their belt, Rogue C...",0
479,Figaro,Andante Dairy,United States,California,"Cow, Goat",NaN,Soft,Microbial,Leaf Wrapped,NaN,Unflavored,"Established in 1999, Andante Dairy is owned an...",0
531,Fuzzy Wheel,Twig Farm,United States,Vermont,"Cow, Goat",NaN,Semi Soft,Animal,Natural,NaN,Unflavored,Twig Farm is a small goat dairy and cheesemaki...,0
541,Gisele,Boston Post Dairy,United States,Vermont,"Cow, Goat",Pasteurised,Firm,Microbial,Washed,Gouda-like,Flavor added to rind,Family-run Boston Post Dairy is set in Enosbur...,0
553,Goatlet,Consider Bardwell / Crown Finish Caves,United States,Vermont / New York,"Cow, Goat",Raw,Firm,Microbial,Natural,NaN,Unflavored,Purchased by Angela Miller and Russell Glove ...,0


In [42]:
culture_chz['Treatment'].unique()

array(['Raw', nan, 'Pasteurised', 'Heat Treated', 'Pasteurised, Raw',
       'Thermalised'], dtype=object)

In [43]:
culture_chz[culture_chz['Name'] == 'Minuet']

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description,Smoked
771,Minuet,Andante Dairy,United States,California,"Goat, Cow",NaN,Soft,Microbial,Mold Ripened,NaN,Unflavored,"Established in 1999, Andante Dairy is owned an...",0


In [44]:
# #culture_chz['Smoked'] = 0
# for i in range(len(culture_chz)):
#     if "Smoked" in culture_chz['Flavor'][i]:
#         culture_chz['Smoked'][i] = 1
#     else:
#         culture_chz['Smoked'][i] = 0

In [45]:
culture_chz

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description,Smoked
0,Abbaye de Belloc,Abbaye de Notre-Dame de Belloc,France,Aquitaine,Sheep,Raw,Firm,Animal,"Natural, Paprika","Alpine-style, Cheddar-like",Flavor added to rind,Abbaye de Belloc is a French cheese that is ma...,0
1,Abbaye de Citeaux,Frère Joel & Frère Frédéric,France,Burgundy,Cow,Raw,Semi Soft,Animal,"Washed, Mold Ripened","Soft-Ripened (Brie-like), Washed-Rind",Unflavored,"Located near Losne in Burgundy, France, Abbaye...",0
2,Abbaye de Tamie,Monks of Tamie,France,Savoie,Cow,Raw,Semi Soft,Animal,Washed,Washed-Rind,Unflavored,"Made from raw cow`s milk, Abbaye de Tamie is p...",0
3,Abbaye du Mont des Cats,Abbaye du Mont des Cats,France,Nord-Pas-de-Calais,Cow,NaN,Semi Soft,Vegetable,Washed,"Alpine-style, Washed-Rind",Unflavored,Abbaye du Mont des Cats is a cow`s milk cheese...,0
4,Abondance,Various,France,Haut Savoie,Cow,Raw,Firm,Animal,Natural,Alpine-style,Unflavored,Named after a valley in the French Alps where ...,0
5,Acapella,Andante Dairy,United States,California,Goat,Pasteurised,Soft,Microbial,"Mold Ripened, Ash coated",Soft-Ripened (Brie-like),Unflavored,"Established in 1999, Andante Dairy is owned an...",0
6,Adams Reserve Cheddar,Great Lakes Cheese,United States,New York,Cow,Pasteurised,Hard,Vegetable,Plasticoat,Cheddar-like,Unflavored,Adams Reserve is produced by Great Lakes Chees...,0
7,Adantino with Tarragon,Andante Dairy,United States,California,Goat,Pasteurised,Soft,Microbial,Mold Ripened,Soft-Ripened (Brie-like),Flavor added to paste,"Established in 1999, Andante Dairy is owned an...",0
8,Aged Cheddar (Grafton),Grafton Village Cheese,United States,Vermont,Cow,Raw,Firm,Microbial,"None, Plastic",Cheddar-like,Unflavored,"Located in Vermont, the Grafton Cooperative Ch...",0
9,Airco,Carr Valley Cheese Co,United States,Wisconsin,"Goat, Sheep, Cow",Pasteurised,Semi Soft,Vegetable,Smoked,"Alpine-style, Cheddar-like",Smoked,Located in the rolling hills of central Wiscon...,1


In [46]:
culture_chz[culture_chz['Producer'] == 'Varies']

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description,Smoked
125,Bonde de Gâtine,Varies,France,Loire,Goat,"Pasteurised, Raw",Firm,Animal,Mold Ripened,Soft-Ripened (Brie-like),Unflavored,"Made in the Loire region of France, the prime ...",0


In [47]:
# Cheese with producer listed as "Varies" should be listed as "Various"
mask = culture_chz['Producer'] == 'Varies'

culture_chz[mask] = culture_chz[mask].replace('Varies', 'Various')

In [48]:
culture_chz['Treatment'].unique()

array(['Raw', nan, 'Pasteurised', 'Heat Treated', 'Pasteurised, Raw',
       'Thermalised'], dtype=object)

In [49]:
culture_chz[culture_chz['Treatment'] == 'Pasteurised, Raw']

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description,Smoked
24,Appenzeller,Alois Pfister,Switzerland,Appenzell,Cow,"Pasteurised, Raw",Firm,Animal,"Washed, Natural",Alpine-style,Unflavored,Produced near the town of Appenzell in northea...,0
29,Arseniko Naxou,Various,Greece,Naxos,"Sheep, Cow, Goat","Pasteurised, Raw",Firm,NaN,"Natural, Flavor Added",NaN,Unflavored,Hailing from the mountain village of Koronos o...,0
106,Bleu d'Auvergne,Various,France,Auvergne,Cow,"Pasteurised, Raw","Blue, Semi Soft",Animal,Natural,Blue,Unflavored,"Named after the ""Department"" in southeastern F...",0
125,Bonde de Gâtine,Various,France,Loire,Goat,"Pasteurised, Raw",Firm,Animal,Mold Ripened,Soft-Ripened (Brie-like),Unflavored,"Made in the Loire region of France, the prime ...",0
212,Cantal,Various,France,Auvergne,Cow,"Pasteurised, Raw",Firm,Animal,Natural,NaN,Unflavored,This tall cylindrical beauty hails from the A...,0
224,Cascaval,Various,Romania,Various,"Sheep, Cow","Pasteurised, Raw",Firm,NaN,"Smoked, Waxed",Pasta Filata (mozzarella-type),Smoked,Cascaval is a Romanian cheese that was traditi...,1
240,Chabichou du Poitou PDO,Various,France,Poitou-Charentes,Goat,"Pasteurised, Raw",Semi Soft,Vegetable,Mold Ripened,Soft-Ripened (Brie-like),Unflavored,Have you ever seen a chèvre as cute as this ti...,0
468,Feta PDO,Various,Greece,Various,"Sheep, Goat","Pasteurised, Raw",Firm,Animal,Rindless,NaN,Unflavored,The poster child of Mediterranean brined chees...,0
556,Gorgonzola Dolce PDO,Various,Italy,Piedmont or Lombardy,Cow,"Pasteurised, Raw",Soft,Animal,Foil Wrap,Blue,Unflavored,Those seeking a blue cheese with a kick might ...,0
690,Langres,Various,France,Champagne-Ardenne,Cow,"Pasteurised, Raw",Semi Soft,Animal,Washed,Washed-Rind,Unflavored,Langres is French washed-rind cheeseproduced n...,0


In [50]:
culture_chz['Country'].unique()

array(['France', 'United States', 'Sweden', 'Canada', nan, 'Switzerland',
       'Australia', 'Ireland', 'Greece', 'Spain', 'Italy', 'Portugal',
       'United Kingdom', 'Netherlands', 'Corsica',
       'Italy / United States', 'Colombia', 'Belgium', 'Germany',
       'Romania', 'North America', 'USA', 'Mexico', 'Denmark', 'Austria',
       'Norway', 'Cyprus', 'Holland', 'England', 'Brazil', 'Bulgaria'],
      dtype=object)

In [51]:
culture_chz[culture_chz['Country'].isnull()]['Region']

14                   Thurgau
90                 Wisconsin
128                  Vermont
230                  Vermont
331                 New York
375                  Vermont
394               California
400                  Vermont
440                 Illinois
449            Massachusetts
455                 New York
485                Louisiana
487            Massachusetts
526                 New York
696               California
707                  Bavaria
727            Massachusetts
733                  Vermont
795                  Vermont
830     Trentino Alto Aldige
842                  Vermont
843                    Maine
860                      NaN
922            Massachusetts
925                     Alej
972                 New York
1011          North Carolina
1073                New York
1090               Wisconsin
1162               Wisconsin
1168              California
1184                 Vermont
1214        Northern Holland
1224                 Vermont
1225          

In [52]:
# based on description of cheese originally labeled as "North America", decide to relist as "United States, Canada"
culture_chz['Description'][296]

'If you were raised outside the Midwest or have never tried a poutine, you might be wondering why there’s so much hype surrounding these shapeless, lumpy masses. Where do they come from, why do they squeak, and what relation do they have to actual aged cheese? Curds are popular in regions that have a long history of cheddarmaking, particularly in the Midwest and in Canada. In essence, they`re the building blocks of cheddar cheese. They start out like most cheeses do: Milk is coagulated into a gelatinous mass that’s cut into little cubes, which are stirred and heated to remove moisture. In cheddarmaking, those curds are formed together, stacked, and kept warm while acidifying and draining (a process called “cheddaring”).The stacked curds are then re-cut into chunks. If they’re destined to become cheddar, these chunks are salted and pressed into a form—but they can also be eaten just like this, as “cheese curds.” Curds are best if eaten the same day they are made, when their protein stra

In [53]:
# Cheese with Country listed as "North America" can be as "Canada, United States"
mask = culture_chz['Country'] == 'North America'

culture_chz[mask] = culture_chz[mask].replace('North America', 'Canada, United States')

In [54]:
# Cheese with Country listed as "USA" can both be  as "United States"
mask = culture_chz['Country'] == 'USA'

culture_chz[mask] = culture_chz[mask].replace('USA', 'United States')

In [55]:
# Cheese with Country listed as "Italy / United States" can both be  as "Italy, United States"
mask = culture_chz['Country'] == 'Italy / United States'

culture_chz[mask] = culture_chz[mask].replace('Italy / United States', 'Italy, United States')

In [56]:
# Cheese with rennet listed as "Vegetable" and "True Vegetable" can  be listed as "Vegetable"
mask = culture_chz['Rennet'] == 'True Vegetable'

culture_chz[mask] = culture_chz[mask].replace('True Vegetable', 'Vegetable')

In [57]:
# Cheese with Country listed as Null, but Region listed as States in America
# can have the Null Values filled with "United States"
mask1 = culture_chz['Region'] == 'Wisconsin'
mask2 = culture_chz['Region'] == 'Vermont'
mask3 = culture_chz['Region'] == 'New York'
mask4 = culture_chz['Region'] == 'California'
mask5 = culture_chz['Region'] == 'Illinois'
mask6 = culture_chz['Region'] == 'Massachusetts'
mask7 = culture_chz['Region'] == 'Louisiana'
mask8 = culture_chz['Region'] == 'Maine'
mask9 = culture_chz['Region'] == 'North Carolina'

mask = mask1 | mask2 | mask3 | mask4 | mask5 | mask6 | mask7 | mask8 | mask9

culture_chz[mask] = culture_chz[mask].fillna({'Country' :'United States'})

In [58]:
# Cheese with Country listed as Null, but Region Thurgau
# can have the Null Values filled with "Switzerland"
mask = culture_chz['Region'] == 'Thurgau'

culture_chz[mask] = culture_chz[mask].fillna({'Country' :'Switzerland'})

In [59]:
# Cheese with Country listed as Null, but Region listed as Bavaria
# can have the Null Values filled with "Germany"
mask = culture_chz['Region'] == 'Bavaria'

culture_chz[mask] = culture_chz[mask].fillna({'Country' :'Germany'})

In [60]:
# Cheese with Country listed as Null, but Region listed as Trentino Alto Aldige
# can have the Null Values filled with "Germany"
# It is also a misspelling of 'Trentino Alto Adige', so may as well fix that too
mask = culture_chz['Region'] == 'Trentino Alto Aldige'

culture_chz[mask] = culture_chz[mask].fillna({'Country' :'Italy'})
culture_chz[mask] = culture_chz[mask].replace('Trentino Alto Aldige', 'Trentino Alto Adige')

In [61]:
# Cheese with Country listed as Null, but Region listed as Northern Holland
# can have the Null Values filled with "Netherlands"
mask = culture_chz['Region'] == 'Northern Holland'

culture_chz[mask] = culture_chz[mask].fillna({'Country' :'Netherlands'})

In [62]:
# Reading description for the cheese with the null value and null country, I saw that both were there
# Country = Croatia, Region = Island of Pag (the cheese is named 'Paski Sir')
culture_chz['Description'][860]

'Located in the Adriatic Sea off the Coast of Croatia, the Island of Pag is a long, wind-swept isle where only the hardiest of people, animals and flora thrive. The island has long been associated with salt production (at least since Roman times) and its unique breed of small sheep, Paska Ovca, whose milk is used to produce Pag’s famous, ancient and award-winning cheese, Paški Sir. These tough little sheep don’t produce much milk, which means that at least 16 sheep are needed to produce just one 4.5 lb wheel of cheese. Pag has hundreds of small sheep farms that make up larger cooperatives, which provide milk to specific cheese producers. The milk that the sheep produce is intensely salty. This comes from the salt deposited on the wild herbs that the sheep graze on by the Pag Bora, the wind produced by the Velebit mountain range on the mainland. This wind catches droplets of water from the Adriatic and dries the water, leaving salt dust, which settles on the island’s vegetation. **Note:

In [63]:
# Cheese with Country and Region listed as Null, is 'Paski Sir'
# can have the Null Values filled with "Croatia" for Country and 'Island of Pag' for region
mask = culture_chz['Name'] == 'Paski Sir'

culture_chz[mask] = culture_chz[mask].fillna({'Country' :'Croatia', 'Region': 'Island of Pag'})

In [64]:
# By reading the description, the Cheese with Region listed as 'Alej' is supposed to be 'Alentejo'
# the country is supposed to be 'Portugal'
culture_chz['Description'][925]

'Produced in the town of Evora in the Alentejo region of southern Portugal, Queijo de Evora is made from raw sheep`s milk.Production runs from November, and peaks in March or April when milk production is at its richest.Flavors reflect the pasture that the sheep are grazing on, as well as the use of thistle rennet as the coagulant during production. Thistle rennet imparts a unique, slightly bitter (but not unpleasant) flavor to cheese that is balanced with salty, fruity notes and a slightly citrus-like finish. With age, these flavors become much more pronounced and assertive.The rind and paste of Queijo de Evora is a delicate, pale yellow and quite thick and meaty. Generally it is recommended that the rind is not eaten.'

In [67]:
# Cheese with Region listed as 'Alej', is supposed to be 'Alentejo'
# the country (which is Null) should be 'Portugal'
mask = culture_chz['Region'] == 'Alej'

culture_chz[mask] = culture_chz[mask].fillna({'Country' :'Portugal'})
culture_chz[mask] = culture_chz[mask].replace('Alej', 'Alentejo')

In [68]:
culture_chz

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description,Smoked
0,Abbaye de Belloc,Abbaye de Notre-Dame de Belloc,France,Aquitaine,Sheep,Raw,Firm,Animal,"Natural, Paprika","Alpine-style, Cheddar-like",Flavor added to rind,Abbaye de Belloc is a French cheese that is ma...,0
1,Abbaye de Citeaux,Frère Joel & Frère Frédéric,France,Burgundy,Cow,Raw,Semi Soft,Animal,"Washed, Mold Ripened","Soft-Ripened (Brie-like), Washed-Rind",Unflavored,"Located near Losne in Burgundy, France, Abbaye...",0
2,Abbaye de Tamie,Monks of Tamie,France,Savoie,Cow,Raw,Semi Soft,Animal,Washed,Washed-Rind,Unflavored,"Made from raw cow`s milk, Abbaye de Tamie is p...",0
3,Abbaye du Mont des Cats,Abbaye du Mont des Cats,France,Nord-Pas-de-Calais,Cow,NaN,Semi Soft,Vegetable,Washed,"Alpine-style, Washed-Rind",Unflavored,Abbaye du Mont des Cats is a cow`s milk cheese...,0
4,Abondance,Various,France,Haut Savoie,Cow,Raw,Firm,Animal,Natural,Alpine-style,Unflavored,Named after a valley in the French Alps where ...,0
5,Acapella,Andante Dairy,United States,California,Goat,Pasteurised,Soft,Microbial,"Mold Ripened, Ash coated",Soft-Ripened (Brie-like),Unflavored,"Established in 1999, Andante Dairy is owned an...",0
6,Adams Reserve Cheddar,Great Lakes Cheese,United States,New York,Cow,Pasteurised,Hard,Vegetable,Plasticoat,Cheddar-like,Unflavored,Adams Reserve is produced by Great Lakes Chees...,0
7,Adantino with Tarragon,Andante Dairy,United States,California,Goat,Pasteurised,Soft,Microbial,Mold Ripened,Soft-Ripened (Brie-like),Flavor added to paste,"Established in 1999, Andante Dairy is owned an...",0
8,Aged Cheddar (Grafton),Grafton Village Cheese,United States,Vermont,Cow,Raw,Firm,Microbial,"None, Plastic",Cheddar-like,Unflavored,"Located in Vermont, the Grafton Cooperative Ch...",0
9,Airco,Carr Valley Cheese Co,United States,Wisconsin,"Goat, Sheep, Cow",Pasteurised,Semi Soft,Vegetable,Smoked,"Alpine-style, Cheddar-like",Smoked,Located in the rolling hills of central Wiscon...,1


In [141]:
count = 0
for i in range(len(culture_chz)):
    if culture_chz['Treatment'].isnull()[i]:
        
        if "thermiz" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Thermalised'
            
        if "thermali" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Thermalised'
            
        if "heat treat" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Heat Treated'
            
        if "unpasteur" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Raw'
            
        if "not pasteurized" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Raw'
            
        if "raw milk" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Raw'
            
        if "raw cow" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Raw'
            
        if "raw sheep" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Raw'
            
        if "raw goat" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Raw'
            
        if " raw " in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Raw'
            
        if "uncooked" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Raw'
            
        if " pasteurized" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Pasteurised'
            
        if " pasteurised" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Pasteurised'
            
        if " pasteuri" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Pasteurised'
            
        if " cooked" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Pasteurised'
            
        if " heated" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Heat Treated'
            
        if " heating" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
            culture_chz['Treatment'][i] = 'Heat Treated'
            
#         if "applying heat" in strip_accents(culture_chz['Description'][i].lower().replace('-', ' ')):
#             #culture_chz['Treatment'][i] = 'Heat Treated'



#             print(culture_chz['Name'][i])
#             print(i)
#             print()
#             print(culture_chz['Description'][i])
#             print()
#             print()
#             count += 1
# print(count)

In [146]:
culture_chz['Treatment'].unique()

array(['Raw', 'Pasteurised', nan, 'Heat Treated', 'Pasteurised, Raw',
       'Thermalised'], dtype=object)

In [207]:
culture_chz[culture_chz['Treatment'] == 'Pasteurised, Raw']

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Flavor,Description,...,Rind_Leaf Wrapped,Rind_Mold Ripened,Rind_Natural,Rind_Paprika,Rind_Plastic,Rind_Plasticoat,Rind_Rindless,Rind_Smoked,Rind_Washed,Rind_Waxed
24,Appenzeller,Alois Pfister,Switzerland,Appenzell,Cow,"Pasteurised, Raw",Firm,Animal,Unflavored,Produced near the town of Appenzell in northea...,...,0,0,1,0,0,0,0,0,1,0
29,Arseniko Naxou,Various,Greece,Naxos,"Sheep, Cow, Goat","Pasteurised, Raw",Firm,NaN,Unflavored,Hailing from the mountain village of Koronos o...,...,0,0,1,0,0,0,0,0,0,0
106,Bleu d'Auvergne,Various,France,Auvergne,Cow,"Pasteurised, Raw","Blue, Semi Soft",Animal,Unflavored,"Named after the ""Department"" in southeastern F...",...,0,0,1,0,0,0,0,0,0,0
125,Bonde de Gâtine,Various,France,Loire,Goat,"Pasteurised, Raw",Firm,Animal,Unflavored,"Made in the Loire region of France, the prime ...",...,0,1,0,0,0,0,0,0,0,0
212,Cantal,Various,France,Auvergne,Cow,"Pasteurised, Raw",Firm,Animal,Unflavored,This tall cylindrical beauty hails from the A...,...,0,0,1,0,0,0,0,0,0,0
224,Cascaval,Various,Romania,Various,"Sheep, Cow","Pasteurised, Raw",Firm,NaN,Smoked,Cascaval is a Romanian cheese that was traditi...,...,0,0,0,0,0,0,0,1,0,1
240,Chabichou du Poitou PDO,Various,France,Poitou-Charentes,Goat,"Pasteurised, Raw",Semi Soft,Vegetable,Unflavored,Have you ever seen a chèvre as cute as this ti...,...,0,1,0,0,0,0,0,0,0,0
468,Feta PDO,Various,Greece,Various,"Sheep, Goat","Pasteurised, Raw",Firm,Animal,Unflavored,The poster child of Mediterranean brined chees...,...,0,0,0,0,0,0,1,0,0,0
556,Gorgonzola Dolce PDO,Various,Italy,Piedmont or Lombardy,Cow,"Pasteurised, Raw",Soft,Animal,Unflavored,Those seeking a blue cheese with a kick might ...,...,0,0,0,0,0,0,0,0,0,0
690,Langres,Various,France,Champagne-Ardenne,Cow,"Pasteurised, Raw",Semi Soft,Animal,Unflavored,Langres is French washed-rind cheeseproduced n...,...,0,0,0,0,0,0,0,0,1,0


In [208]:
# used to be raw, now is heated
culture_chz['Description'][24]

'Produced near the town of Appenzell in northeastern Switzerland, Appenzeller is made from cow`s milk and has a history dating back more than 700 years. Appenzeller is a wonderful example of traditional, Swiss mountain cheesemaking at its best. There are now just three local cooperative dairies producing Appenzeller, and milk comes exclusively from the Simmenthaler breed of cow. Traditionally cheeses were only made using raw milk, however, the rules have become less stringent and now some dairies prefer to pasteurize. As with many Swiss mountain cheeses, once the curd is formed and cut, it is then reheated to a higher temperature to expel more whey from the solids, forming smaller pieces of curd in the process. The curd is then pressed while still in the vat, submerged under the whey before being transferred to molds for further pressing. After unmolding, wheels are moved to the maturing rooms where they are aged for 7-12 months. One of the distinctive steps in the maturing process of 

In [151]:
culture_chz['Milk'].unique()

array(['Sheep', 'Cow', 'Goat', 'Goat, Sheep, Cow', 'Sheep, Cow, Goat',
       'Sheep, Goat', 'Buffalo', 'Sheep, Cow', 'Goat, Cow', 'Goat, Sheep',
       'Cow, Goat', 'Sheep, Goat, Cow', 'Buffalo, Cow', 'Cow, Sheep',
       'Cow, Buffalo'], dtype=object)

In [152]:
culture_chz['Style'].unique()

array(['Alpine-style, Cheddar-like',
       'Soft-Ripened (Brie-like), Washed-Rind', 'Washed-Rind',
       'Alpine-style, Washed-Rind', 'Alpine-style',
       'Soft-Ripened (Brie-like)', 'Cheddar-like',
       'Alpine-style, Gouda-like, Washed-Rind',
       'Washed-Rind, Alpine-style', 'Blue', 'Grana / Grating cheese', nan,
       'Fresh', 'Alpine-style, Soft-Ripened (Brie-like)',
       'Pasta Filata (mozzarella-type)',
       'Alpine-style, Cheddar-like, Grana / Grating cheese',
       'Crème, Washed-Rind',
       'Cheddar-like, Gouda-like, Grana / Grating cheese, Washed-Rind',
       'Blue, Cheddar-like', 'Gouda-like, Grana / Grating cheese',
       'Cheddar-like, Washed-Rind',
       'Cheddar-like, Grana / Grating cheese',
       'Blue, Soft-Ripened (Brie-like)',
       'Alpine-style, Cheddar-like, Gouda-like', 'Alpine-style, Blue',
       'Gouda-like', 'Crème, Soft-Ripened (Brie-like)',
       'Fresh, Pasta Filata (mozzarella-type)', 'Crème', 'Blue, Crème',
       'Feta-like', 'Al

In [153]:
eda(culture_chz)

missing values: 
Name                0
Producer           31
Country             0
Region             11
Milk                0
Treatment         313
Classification     18
Rennet             42
Rind               30
Style             699
Flavor              0
Description         0
Smoked              0
dtype: int64

dataframe index: 
RangeIndex(start=0, stop=1244, step=1)

dataframe types: 
Name              object
Producer          object
Country           object
Region            object
Milk              object
Treatment         object
Classification    object
Rennet            object
Rind              object
Style             object
Flavor            object
Description       object
Smoked             int64
dtype: object

dataframe shape: 
(1244, 13)

dataframe describe: 
            Smoked
count  1244.000000
mean      0.017685
std       0.131856
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max       1.000000

number of unique values in each column: 
Nam

In [155]:
culture_chz.head()

,Name,Producer,Country,Region,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description,Smoked
0,Abbaye de Belloc,Abbaye de Notre-Dame de Belloc,France,Aquitaine,Sheep,Raw,Firm,Animal,"Natural, Paprika","Alpine-style, Cheddar-like",Flavor added to rind,Abbaye de Belloc is a French cheese that is ma...,0
1,Abbaye de Citeaux,Frère Joel & Frère Frédéric,France,Burgundy,Cow,Raw,Semi Soft,Animal,"Washed, Mold Ripened","Soft-Ripened (Brie-like), Washed-Rind",Unflavored,"Located near Losne in Burgundy, France, Abbaye...",0
2,Abbaye de Tamie,Monks of Tamie,France,Savoie,Cow,Raw,Semi Soft,Animal,Washed,Washed-Rind,Unflavored,"Made from raw cow`s milk, Abbaye de Tamie is p...",0
3,Abbaye du Mont des Cats,Abbaye du Mont des Cats,France,Nord-Pas-de-Calais,Cow,Pasteurised,Semi Soft,Vegetable,Washed,"Alpine-style, Washed-Rind",Unflavored,Abbaye du Mont des Cats is a cow`s milk cheese...,0
4,Abondance,Various,France,Haut Savoie,Cow,Raw,Firm,Animal,Natural,Alpine-style,Unflavored,Named after a valley in the French Alps where ...,0


In [187]:
# save progress

#save progress in csv
#culture_chz.to_csv('./data/culture_model_testing.csv', index=False)

In [201]:
#read culture_model_testing.csv into 'culture_chz'
culture_chz = pd.read_csv('./data/culture_model_testing.csv')


In [202]:
# note that this function is slightly altered from the one for cheese.com dataset

def dummy_listed_features(df, feature):

    # for ceratin features, some cells have multiple values
    # to get all of the values of these features, first create a list 
    # of all of the UNIQUE values that appear in the selected feature

    # fill the null values of the feature with a string value of "None" so as to not throw an error
    df[feature].fillna('None', inplace=True)

    # create an empty string
    strang = ""
    # look through every cell of the feature
    for i in df[feature].unique():
        # add every entry in the cell into a long string seperated by a comma and a space
        strang += i + ', '

    # strip the extraneous comma and whitespace from the end of the newly gathered string
    # replace ' and' with a comma
    strang = strang.replace(' and', ',').strip(', ')

    # seperate the long string of values by a comma and space into a list of values in the feature
    # convert this list into a set to drop the values that appear more then once in an entry for the feature
    # convert this back into a list so it can be indexed 
    feature_list = sorted(set(strang.split(', ')))

    # initialize a dummy column for each unique value of the feature
    for i in feature_list:
        # initialize each column as 0
        df[f'{feature}_{i}']=0

    # go through every cheese in the list
    for i in range(len(df)):

        # look at each value in each cell by seperating by commas, putting it into a list, 
        # and iterating through that list (remember to replace any ' and' with a comma before creating list)
        for k in df[feature][i].replace(' and', ',').split(', '):
            # if the value exists in the feature for a particular cheese, set the dummy value to 1
            df[f'{feature}_{k}'][i] = 1
            
    # once feature has been dummied, drop the original        
    df.drop(columns=feature, inplace=True)

    # also drop the feature column created when the value was 'None' because that was a null value before        
    df.drop(columns=f'{feature}_None', inplace=True)
    
    print(f'List of values for {feature} : ')
    print(feature_list)     

In [203]:
dummy_listed_features(culture_chz, 'Style')

List of values for Style : 
['Alpine-style', 'Blue', 'Cheddar-like', 'Crème', 'Feta-like', 'Fresh', 'Gouda-like', 'Grana / Grating cheese', 'None', 'Pasta Filata (mozzarella-type)', 'Soft-Ripened (Brie-like)', 'Washed-Rind']


/Users/michaelknight/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [204]:
dummy_listed_features(culture_chz, 'Rind')

List of values for Rind : 
['Ash', 'Ash coated', 'Ashed', 'Blue', 'Cloth Wrapped', 'Cocoa', 'Flavor Added', 'Flavored', 'Foil Wrap', 'Fresh', 'Geotricum', 'Herbed', 'Leaf Wrapped', 'Mold Ripened', 'Natural', 'None', 'Paprika', 'Plastic', 'Plasticoat', 'Rindless', 'Smoked', 'Washed', 'Waxed']


/Users/michaelknight/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [206]:
culture_chz['Treatment'].unique()

array(['Raw', 'Pasteurised', nan, 'Heat Treated', 'Pasteurised, Raw',
       'Thermalised'], dtype=object)